In [1]:
import pandas as pd

df = pd.read_csv('thesis_titles.csv', index_col=0)
df.head()

,author,title,year
0,Jean Spanjersberg,De institutionele groei van het Nederlandse on...,1983
1,Dick de Winter,Disciplinering van protestgedrag van havenarbe...,1983
2,Christi Boerdam,Ongehuwde moeders te Rotterdam in het laatste ...,1983
3,B W H Guyt,De visserij op de Noordzee : de lage landen aa...,1983
4,Harma Tent,Het beeld van de Amerikaanse indiaan in Iberis...,1983


In [107]:
import textrazor
import json

all_data = []
places = []

for index, row in df.iterrows():
    if index <= 490:
        textrazor.api_key = "0c865c935fc553335ca52457e31aa2288972a77b1b4ca3d22f8f5cad"
        client = textrazor.TextRazor(extractors=["entities", "topics"])
        client.set_enrichment_queries(["fbase:/location/location/geolocation>/location/geocode/latitude", "fbase:/location/location/geolocation>/location/geocode/longitude"])

        response = client.analyze(row['title'])
        all_data.append([response.json])

        seen = set()
        p = []
        for entity in response.entities():
            try:
                if entity.data:
                    place = entity.json['entityEnglishId'] if entity.json['entityEnglishId'] else entity.json['entityId']
                    if entity.id not in seen:
                        p.append([place, entity.json['data']['fbase:/location/location/geolocation>/location/geocode/longitude'][0], entity.json['data']['fbase:/location/location/geolocation>/location/geocode/latitude'][0]])
                        seen.add(entity.id)
            except:
                pass
        places.append(p)
    else:
        textrazor.api_key = "da962cf743f78a99046b308f3611abe90346d2ccc6db5141c732fe5e"
        client = textrazor.TextRazor(extractors=["entities", "topics"])
        client.set_enrichment_queries(["fbase:/location/location/geolocation>/location/geocode/latitude", "fbase:/location/location/geolocation>/location/geocode/longitude"])

        response = client.analyze(row['title'])
        all_data.append([response.json])

        seen = set()
        p = []
        for entity in response.entities():
            try:
                if entity.data:
                    place = entity.json['entityEnglishId'] if entity.json['entityEnglishId'] else entity.json['entityId']
                    if entity.id not in seen:
                        p.append([place, entity.json['data']['fbase:/location/location/geolocation>/location/geocode/longitude'][0], entity.json['data']['fbase:/location/location/geolocation>/location/geocode/latitude'][0]])
                        seen.add(entity.id)
            except:
                pass
        places.append(p)
        
with open('all_data.json', 'w') as fp:
    json.dump(all_data, fp)

pl = pd.Series(places)
df['places'] = pl.values
df.to_csv('thesis_titles_with_toponyms.csv')

In [108]:
df.head()

,author,title,year,places
0,Jean Spanjersberg,De institutionele groei van het Nederlandse on...,1983,[]
1,Dick de Winter,Disciplinering van protestgedrag van havenarbe...,1983,[]
2,Christi Boerdam,Ongehuwde moeders te Rotterdam in het laatste ...,1983,"[[Rotterdam, 4.479167, 51.930833]]"
3,B W H Guyt,De visserij op de Noordzee : de lage landen aa...,1983,"[[North Sea, 3.0, 56.0]]"
4,Harma Tent,Het beeld van de Amerikaanse indiaan in Iberis...,1983,"[[United States, -100.0, 40.0]]"
